In [ ]:
!pip install langchain openai faiss-cpu pandas tiktoken langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
import os, json
import pandas as pd
from datetime import datetime
from getpass import getpass
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
from langchain.agents import Tool, initialize_agent, AgentType

from getpass import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0.3)


RULES_PATH = "rules.json"
if not os.path.exists(RULES_PATH):
    with open(RULES_PATH, 'w') as f:
        json.dump({"air_flow": 0.5, "temperature": 30.0}, f)
with open(RULES_PATH) as f:
    rules = json.load(f)

def rule_based_diagnosis(data):
    r = rules
    if data["air_flow"] < r["air_flow"] and not data["compressor_status"]:
        return "Fault", "Low airflow + compressor OFF", "Low"
    if data["temperature"] > r["temperature"] and not data["fan_status"]:
        return "Fault", "High temp + fan OFF", "Medium"
    return "No Fault", "Normal", "High"


if not os.path.exists("faiss_memory"):
    db = FAISS.from_texts(["seed"], OpenAIEmbeddings())
    db.save_local("faiss_memory")
db = FAISS.load_local("faiss_memory", OpenAIEmbeddings())
memory = VectorStoreRetrieverMemory(retriever=db.as_retriever())


def actuator_control(action):
    return f"[Actuator] {action} simulated."

def update_rule(json_str):
    update = json.loads(json_str)
    rules.update(update)
    with open(RULES_PATH, 'w') as f:
        json.dump(rules, f)
    return f"Rule updated: {update}"

def store_case(input_data):
    db.add_texts([json.dumps(input_data)])
    return "Case stored."

tools = [
    Tool("ActuatorControl", actuator_control, "Simulate control actions"),
    Tool("RuleUpdater", update_rule, "Update the rulebook live"),
    Tool("MemoryStore", store_case, "Store input in FAISS memory")
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, memory=memory)

prompt = PromptTemplate.from_template("""
You're an autonomous HVAC agent.
Input:
{sensor_data}
Return JSON: ["Diagnosis", "Reason", "Confidence", "Recommended Action", "Rule Suggestion"]
""")
chain = LLMChain(llm=llm, prompt=prompt)

logfile = "diagnostic_log.csv"
if not os.path.exists(logfile):
    with open(logfile, 'w') as f:
        f.write("timestamp,rule_diagnosis,gpt_diagnosis,ground_truth,correct,gpt_confidence,action,rule_suggestion\n")

def run_agent_on_row(row, gt):
    input_data = {
        "temperature": float(row.get("Zone Temperature", 25.0)),
        "humidity": float(row.get("Zone Humidity", 50.0)),
        "air_flow": float(row.get("Supply Air Flow Rate", 1.0)),
        "fan_status": row.get("Fan Status", 1.0) > 0,
        "compressor_status": row.get("Compressor Status", 1.0) > 0
    }
    rule_diag, _, _ = rule_based_diagnosis(input_data)
    result = chain.run(sensor_data=json.dumps(input_data))
    try:
        parsed = json.loads(result)
    except:
        parsed = {"Diagnosis": "Parse Error", "Confidence": "Low", "Recommended Action": "None", "Rule Suggestion": "{}"}

    if parsed["Recommended Action"] != "None":
        agent.run(f"ActuatorControl: {parsed['Recommended Action']}")
    if parsed["Rule Suggestion"] != "{}":
        agent.run(f"RuleUpdater: {parsed['Rule Suggestion']}")
    agent.run(f"MemoryStore: {json.dumps(input_data)}")

    correct = parsed["Diagnosis"] == gt
    with open(logfile, 'a') as f:
        f.write(f"{datetime.now()},{rule_diag},{parsed['Diagnosis']},{gt},{correct},{parsed['Confidence']},{parsed['Recommended Action']},{parsed['Rule Suggestion']}\n")


def run_on_csv(csv_path):
    df = pd.read_csv(csv_path)
    if "Fault Detection Ground Truth" not in df.columns:
        print("Missing ground truth column.")
        return
    df = df.dropna(subset=["Fault Detection Ground Truth"])
    df = df.sample(n=min(25, len(df)))
    for _, row in df.iterrows():
        label = "Fault" if row["Fault Detection Ground Truth"] == 1 else "No Fault"
        run_agent_on_row(row, label)
    print("Completed run. Logs saved.")